Part 2 - Section A - Sentiment Prediction

In [128]:
pip install emoji

In [129]:
#importing relevant libraries
import pandas as pd 
import numpy as np 
from emoji import UNICODE_EMOJI
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import string
analyser = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [130]:
#read data file
tweets = pd.read_csv('data.csv' ,encoding="utf-8")
len(tweets)

513

In [131]:
#printing file
tweets.head(10)

,tweet_id,text
0,1,. I really love the song. So relatable @ALIEGA...
1,2,People should start watching the not mainstrea...
2,3,I am so happy I am able to bring phone to my s...
3,4,The actors in#PeakyBlinders are too good. I am...
4,5,The way @RyanMcMullan @ryavanhhuu__ sings are ...
5,6,CANT WAIT To WATCH @RIVERDALE NEW SEASON! \xF0...
6,7,him @CHRISPRATFANFOREVER45 is soo funny !!! I ...
7,8,All language films are good. I like exploring ...
8,9,this @13Reasonswhy shouldnâ€™t end so soon. IM...
9,10,I need a break from all these social media mes...


In [137]:
#Emoji sentiment database
emojisentiment = pd.read_excel('emojisentiment.xlsx')
emojisentiment = pd.read_excel('emojisentiment.xlsx',usecols=['emoji', 'sentiment'])
print(emojisentiment.to_dict(orient='record'))

[{'emoji': '\\xF0\\x9F\\x98\\x82', 'sentiment': 0.221}, {'emoji': '\\xF0\\x9F\\x98\\x8D', 'sentiment': 0.678}, {'emoji': '\\xF0\\x9F\\x98\\x8A', 'sentiment': 0.644}, {'emoji': '\\xF0\\x9F\\x98\\xAD', 'sentiment': -0.093}, {'emoji': '\\xF0\\x9F\\x98\\xA9', 'sentiment': -0.368}, {'emoji': '\\xF0\\x9F\\x98\\x92', 'sentiment': -0.374}, {'emoji': '\\xF0\\x9F\\x98\\xB3', 'sentiment': 0.018}, {'emoji': '\\xF0\\x9F\\x98\\xA1', 'sentiment': -0.173}, {'emoji': '\\xF0\\x9F\\x98\\xB1', 'sentiment': 0.19}, {'emoji': 'dY~,', 'sentiment': 0.221}, {'emoji': 'dY~_', 'sentiment': 0.678}, {'emoji': 'dY~T', 'sentiment': 0.644}, {'emoji': 'dY~-', 'sentiment': -0.093}, {'emoji': 'dY~c', 'sentiment': -0.368}, {'emoji': "dY~'", 'sentiment': -0.374}, {'emoji': 'dY~3', 'sentiment': 0.018}, {'emoji': 'dY~\xad', 'sentiment': -0.173}, {'emoji': 'dY~ñ', 'sentiment': 0.19}, {'emoji': ':face_with_tears_of_joy:', 'sentiment': 0.221}, {'emoji': ':smiling_face_with_heart-eyes:', 'sentiment': 0.678}, {'emoji': ':smiling_

In [ ]:
#0  \xF0\x9F\x98\x82    FACE WITH TEARS OF JOY 😂 dY~,
#1  \xF0\x9F\x98\x8D    SMILING FACE WITH HEART-SHAPED EYES 😍 dY~_
#2  \xF0\x9F\x98\x8A    SMILING FACE WITH SMILING EYES 😊 dY~T
#3  \xF0\x9F\x98\xAD    LOUDLY CRYING FACE 😭 dY~-
#4  \xF0\x9F\x98\xA9    WEARY FACE 😩 dY~c
#5  \xF0\x9F\x98\x92    UNAMUSED FACE 😒 dY~'
#6  \xF0\x9F\x98\xB3    FLUSHED FACE 😳 dY~3
#7  \xF0\x9F\x98\xA1    POUTING FACE 😡 dY~­
#8  \xF0\x9F\x98\xB1    FACE SCREAMING IN FEAR 😱


In [133]:
#Tokenization the text
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize                      

data = str(tweets['text'])
nltk_tokens = nltk.word_tokenize(data)

tweets['tokens'] = (tweets['text']).apply(lambda x: [i for i in x.split()])       
tweets.head(20)
   

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,tweet_id,text,tokens
0,1,. I really love the song. So relatable @ALIEGA...,"[., I, really, love, the, song., So, relatable..."
1,2,People should start watching the not mainstrea...,"[People, should, start, watching, the, not, ma..."
2,3,I am so happy I am able to bring phone to my s...,"[I, am, so, happy, I, am, able, to, bring, pho..."
3,4,The actors in#PeakyBlinders are too good. I am...,"[The, actors, in#PeakyBlinders, are, too, good..."
4,5,The way @RyanMcMullan @ryavanhhuu__ sings are ...,"[The, way, @RyanMcMullan, @ryavanhhuu__, sings..."
5,6,CANT WAIT To WATCH @RIVERDALE NEW SEASON! \xF0...,"[CANT, WAIT, To, WATCH, @RIVERDALE, NEW, SEASO..."
6,7,him @CHRISPRATFANFOREVER45 is soo funny !!! I ...,"[him, @CHRISPRATFANFOREVER45, is, soo, funny, ..."
7,8,All language films are good. I like exploring ...,"[All, language, films, are, good., I, like, ex..."
8,9,this @13Reasonswhy shouldnâ€™t end so soon. IM...,"[this, @13Reasonswhy, shouldnâ€™t, end, so, so..."
9,10,I need a break from all these social media mes...,"[I, need, a, break, from, all, these, social, ..."


In [134]:
#Finding emoji 
emojisentiment = dict(emojisentiment.values.tolist())  #Converting to dicts
i = 0
all_tweets = list(tweets['text'])
emoji = [] #empty series to hold our predicted values
for tweet in all_tweets:
  nltk_tokens = nltk.word_tokenize(str(tweet))
  print(nltk_tokens)
  for token in nltk_tokens:
    if token in emojisentiment.keys():
      print(token)
      e= token
  emoji.append(e)

tweets['emoji'] = emoji 


['.', 'I', 'really', 'love', 'the', 'song', '.', 'So', 'relatable', '@', 'ALIEGATIE', 'LOVE', '\\xF0\\x9F\\x98\\x82', '\\xF0\\x9F\\x98\\x8D']
\xF0\x9F\x98\x82
\xF0\x9F\x98\x8D
['People', 'should', 'start', 'watching', 'the', 'not', 'mainstream', 'movies', '\\xF0\\x9F\\x98\\x8A']
\xF0\x9F\x98\x8A
['I', 'am', 'so', 'happy', 'I', 'am', 'able', 'to', 'bring', 'phone', 'to', 'my', 'school.\\xF0\\x9F\\x98\\x82', '@', 'SHARKISes_', 'are', "n't", 'we', 'happy', 'about', 'it', '?', '?', '?', '?', '\\xF0\\x9F\\x98\\xB3']
\xF0\x9F\x98\xB3
['The', 'actors', 'in', '#', 'PeakyBlinders', 'are', 'too', 'good', '.', 'I', 'am', 'excited', 'to', 'the', 'next', 'season', '#', 'excited', '\\xF0\\x9F\\x98\\x8A']
\xF0\x9F\x98\x8A
['The', 'way', '@', 'RyanMcMullan', '@', 'ryavanhhuu__', 'sings', 'are', 'so', 'heavenly', '.', 'People', 'listen', 'to', 'him', '#', 'waytooggo', '\\xF0\\x9F\\x98\\x8A']
\xF0\x9F\x98\x8A
['CANT', 'WAIT', 'To', 'WATCH', '@', 'RIVERDALE', 'NEW', 'SEASON', '!', '\\xF0\\x9F\\x98\\x8A',

In [135]:
tweets.head()

,tweet_id,text,tokens,emoji
0,1,. I really love the song. So relatable @ALIEGA...,"[., I, really, love, the, song., So, relatable...",\xF0\x9F\x98\x8D
1,2,People should start watching the not mainstrea...,"[People, should, start, watching, the, not, ma...",\xF0\x9F\x98\x8A
2,3,I am so happy I am able to bring phone to my s...,"[I, am, so, happy, I, am, able, to, bring, pho...",\xF0\x9F\x98\xB3
3,4,The actors in#PeakyBlinders are too good. I am...,"[The, actors, in#PeakyBlinders, are, too, good...",\xF0\x9F\x98\x8A
4,5,The way @RyanMcMullan @ryavanhhuu__ sings are ...,"[The, way, @RyanMcMullan, @ryavanhhuu__, sings...",\xF0\x9F\x98\x8A


In [138]:
#Finding emoji score
emojisentiment = dict(emojisentiment.values.tolist())  #Converting to dicts
analyser = SentimentIntensityAnalyzer()
all_tweets = list(tweets['text'])
emoji_sentimental_score = []
for tweet in all_tweets:
   nltk_tokens = nltk.word_tokenize(str(tweet))
   print(nltk_tokens)
   avrg_sentimental_score = 0.0
   score = 0
   counter = 0
   for token in nltk_tokens:
       if token in emojisentiment.keys():
           score += emojisentiment[token]
           counter += 1
   if counter > 0:
       avrg_sentimental_score = score/counter
   emoji_sentimental_score.append(avrg_sentimental_score)

tweets['emoji_score'] = emoji_sentimental_score      
print(tweets[tweets['emoji_score'] > 0])


['.', 'I', 'really', 'love', 'the', 'song', '.', 'So', 'relatable', '@', 'ALIEGATIE', 'LOVE', '\\xF0\\x9F\\x98\\x82', '\\xF0\\x9F\\x98\\x8D']
['People', 'should', 'start', 'watching', 'the', 'not', 'mainstream', 'movies', '\\xF0\\x9F\\x98\\x8A']
['I', 'am', 'so', 'happy', 'I', 'am', 'able', 'to', 'bring', 'phone', 'to', 'my', 'school.\\xF0\\x9F\\x98\\x82', '@', 'SHARKISes_', 'are', "n't", 'we', 'happy', 'about', 'it', '?', '?', '?', '?', '\\xF0\\x9F\\x98\\xB3']
['The', 'actors', 'in', '#', 'PeakyBlinders', 'are', 'too', 'good', '.', 'I', 'am', 'excited', 'to', 'the', 'next', 'season', '#', 'excited', '\\xF0\\x9F\\x98\\x8A']
['The', 'way', '@', 'RyanMcMullan', '@', 'ryavanhhuu__', 'sings', 'are', 'so', 'heavenly', '.', 'People', 'listen', 'to', 'him', '#', 'waytooggo', '\\xF0\\x9F\\x98\\x8A']
['CANT', 'WAIT', 'To', 'WATCH', '@', 'RIVERDALE', 'NEW', 'SEASON', '!', '\\xF0\\x9F\\x98\\x8A', '@', 'colesprouse']
['him', '@', 'CHRISPRATFANFOREVER45', 'is', 'soo', 'funny', '!', '!', '!', 'I', '

In [139]:
tweets.head(10)

,tweet_id,text,tokens,emoji,emoji_score
0,1,. I really love the song. So relatable @ALIEGA...,"[., I, really, love, the, song., So, relatable...",\xF0\x9F\x98\x8D,0.4495
1,2,People should start watching the not mainstrea...,"[People, should, start, watching, the, not, ma...",\xF0\x9F\x98\x8A,0.6440
2,3,I am so happy I am able to bring phone to my s...,"[I, am, so, happy, I, am, able, to, bring, pho...",\xF0\x9F\x98\xB3,0.0180
3,4,The actors in#PeakyBlinders are too good. I am...,"[The, actors, in#PeakyBlinders, are, too, good...",\xF0\x9F\x98\x8A,0.6440
4,5,The way @RyanMcMullan @ryavanhhuu__ sings are ...,"[The, way, @RyanMcMullan, @ryavanhhuu__, sings...",\xF0\x9F\x98\x8A,0.6440
5,6,CANT WAIT To WATCH @RIVERDALE NEW SEASON! \xF0...,"[CANT, WAIT, To, WATCH, @RIVERDALE, NEW, SEASO...",\xF0\x9F\x98\x8A,0.6440
6,7,him @CHRISPRATFANFOREVER45 is soo funny !!! I ...,"[him, @CHRISPRATFANFOREVER45, is, soo, funny, ...",\xF0\x9F\x98\x82,0.2210
7,8,All language films are good. I like exploring ...,"[All, language, films, are, good., I, like, ex...",\xF0\x9F\x98\xB3,0.0180
8,9,this @13Reasonswhy shouldnâ€™t end so soon. IM...,"[this, @13Reasonswhy, shouldnâ€™t, end, so, so...",\xF0\x9F\x98\x8A,0.6440
9,10,I need a break from all these social media mes...,"[I, need, a, break, from, all, these, social, ...",\xF0\x9F\x98\x8A,0.0000


In [140]:
#Remove stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
additional  = ['rt','rts','retweet']
swords = set().union(stopwords.words('english'),additional)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [141]:
#Remove username, hastags, links, numbers
tweets['preprocessed_text'] = tweets['text'].str.lower()\
          .str.replace('(@[a-z0-9]+)\w+',' ')\
          .str.replace('(x[a-z0-9]+)\w+',' ')\
          .str.replace('(http\S+)', ' ')\
          .str.replace('([^0-9a-z \t])',' ')\
          .str.replace(' +',' ')\
          .apply(lambda x: [i for i in x.split() if not i in swords])

tweets.head(10)

,tweet_id,text,tokens,emoji,emoji_score,preprocessed_text
0,1,. I really love the song. So relatable @ALIEGA...,"[., I, really, love, the, song., So, relatable...",\xF0\x9F\x98\x8D,0.4495,"[really, love, song, relatable, love]"
1,2,People should start watching the not mainstrea...,"[People, should, start, watching, the, not, ma...",\xF0\x9F\x98\x8A,0.6440,"[people, start, watching, mainstream, movies]"
2,3,I am so happy I am able to bring phone to my s...,"[I, am, so, happy, I, am, able, to, bring, pho...",\xF0\x9F\x98\xB3,0.0180,"[happy, able, bring, phone, school, happy]"
3,4,The actors in#PeakyBlinders are too good. I am...,"[The, actors, in#PeakyBlinders, are, too, good...",\xF0\x9F\x98\x8A,0.6440,"[actors, peakyblinders, good, e, next, season, e]"
4,5,The way @RyanMcMullan @ryavanhhuu__ sings are ...,"[The, way, @RyanMcMullan, @ryavanhhuu__, sings...",\xF0\x9F\x98\x8A,0.6440,"[way, sings, heavenly, people, listen, waytooggo]"
5,6,CANT WAIT To WATCH @RIVERDALE NEW SEASON! \xF0...,"[CANT, WAIT, To, WATCH, @RIVERDALE, NEW, SEASO...",\xF0\x9F\x98\x8A,0.6440,"[cant, wait, watch, new, season]"
6,7,him @CHRISPRATFANFOREVER45 is soo funny !!! I ...,"[him, @CHRISPRATFANFOREVER45, is, soo, funny, ...",\xF0\x9F\x98\x82,0.2210,"[soo, funny, died, laughing, hahhahahahhah]"
7,8,All language films are good. I like exploring ...,"[All, language, films, are, good., I, like, ex...",\xF0\x9F\x98\xB3,0.0180,"[language, films, good, like, e]"
8,9,this @13Reasonswhy shouldnâ€™t end so soon. IM...,"[this, @13Reasonswhy, shouldnâ€™t, end, so, so...",\xF0\x9F\x98\x8A,0.6440,"[end, soon, imiss]"
9,10,I need a break from all these social media mes...,"[I, need, a, break, from, all, these, social, ...",\xF0\x9F\x98\x8A,0.0000,"[need, break, social, media, mess, ever, done]"


In [142]:
#Importing lexicon for sentiment polarity
import nltk.sentiment.vader as vd
from nltk import download
download('vader_lexicon')
sia = vd.SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [143]:
#Calculate sentiment score
from nltk.tokenize import word_tokenize
tweets['sentiment_score'] = tweets['preprocessed_text'].apply(lambda x: sum([ sia.polarity_scores(i)['compound'] for i in word_tokenize( ' '.join(x) )]) )
tweets.head(10)

,tweet_id,text,tokens,emoji,emoji_score,preprocessed_text,sentiment_score
0,1,. I really love the song. So relatable @ALIEGA...,"[., I, really, love, the, song., So, relatable...",\xF0\x9F\x98\x8D,0.4495,"[really, love, song, relatable, love]",1.2738
1,2,People should start watching the not mainstrea...,"[People, should, start, watching, the, not, ma...",\xF0\x9F\x98\x8A,0.6440,"[people, start, watching, mainstream, movies]",0.0000
2,3,I am so happy I am able to bring phone to my s...,"[I, am, so, happy, I, am, able, to, bring, pho...",\xF0\x9F\x98\xB3,0.0180,"[happy, able, bring, phone, school, happy]",1.1438
3,4,The actors in#PeakyBlinders are too good. I am...,"[The, actors, in#PeakyBlinders, are, too, good...",\xF0\x9F\x98\x8A,0.6440,"[actors, peakyblinders, good, e, next, season, e]",0.4404
4,5,The way @RyanMcMullan @ryavanhhuu__ sings are ...,"[The, way, @RyanMcMullan, @ryavanhhuu__, sings...",\xF0\x9F\x98\x8A,0.6440,"[way, sings, heavenly, people, listen, waytooggo]",0.6124
5,6,CANT WAIT To WATCH @RIVERDALE NEW SEASON! \xF0...,"[CANT, WAIT, To, WATCH, @RIVERDALE, NEW, SEASO...",\xF0\x9F\x98\x8A,0.6440,"[cant, wait, watch, new, season]",0.0000
6,7,him @CHRISPRATFANFOREVER45 is soo funny !!! I ...,"[him, @CHRISPRATFANFOREVER45, is, soo, funny, ...",\xF0\x9F\x98\x82,0.2210,"[soo, funny, died, laughing, hahhahahahhah]",0.3769
7,8,All language films are good. I like exploring ...,"[All, language, films, are, good., I, like, ex...",\xF0\x9F\x98\xB3,0.0180,"[language, films, good, like, e]",0.8016
8,9,this @13Reasonswhy shouldnâ€™t end so soon. IM...,"[this, @13Reasonswhy, shouldnâ€™t, end, so, so...",\xF0\x9F\x98\x8A,0.6440,"[end, soon, imiss]",0.0000
9,10,I need a break from all these social media mes...,"[I, need, a, break, from, all, these, social, ...",\xF0\x9F\x98\x8A,0.0000,"[need, break, social, media, mess, ever, done]",-0.3612


In [144]:
#Add text score and emoji score
i = 0
final_sentiment = [ ]  #empty series to hold our final sentiment values

while(i<len(tweets)):
    y = 0
    y = (tweets.iloc[i]['sentiment_score']) + (tweets.iloc[i]['emoji_score'])
    final_sentiment.append(y)
    i = i+1

tweets['final_sentimentvalue'] = final_sentiment
tweets.head(10)


,tweet_id,text,tokens,emoji,emoji_score,preprocessed_text,sentiment_score,final_sentimentvalue
0,1,. I really love the song. So relatable @ALIEGA...,"[., I, really, love, the, song., So, relatable...",\xF0\x9F\x98\x8D,0.4495,"[really, love, song, relatable, love]",1.2738,1.7233
1,2,People should start watching the not mainstrea...,"[People, should, start, watching, the, not, ma...",\xF0\x9F\x98\x8A,0.6440,"[people, start, watching, mainstream, movies]",0.0000,0.6440
2,3,I am so happy I am able to bring phone to my s...,"[I, am, so, happy, I, am, able, to, bring, pho...",\xF0\x9F\x98\xB3,0.0180,"[happy, able, bring, phone, school, happy]",1.1438,1.1618
3,4,The actors in#PeakyBlinders are too good. I am...,"[The, actors, in#PeakyBlinders, are, too, good...",\xF0\x9F\x98\x8A,0.6440,"[actors, peakyblinders, good, e, next, season, e]",0.4404,1.0844
4,5,The way @RyanMcMullan @ryavanhhuu__ sings are ...,"[The, way, @RyanMcMullan, @ryavanhhuu__, sings...",\xF0\x9F\x98\x8A,0.6440,"[way, sings, heavenly, people, listen, waytooggo]",0.6124,1.2564
5,6,CANT WAIT To WATCH @RIVERDALE NEW SEASON! \xF0...,"[CANT, WAIT, To, WATCH, @RIVERDALE, NEW, SEASO...",\xF0\x9F\x98\x8A,0.6440,"[cant, wait, watch, new, season]",0.0000,0.6440
6,7,him @CHRISPRATFANFOREVER45 is soo funny !!! I ...,"[him, @CHRISPRATFANFOREVER45, is, soo, funny, ...",\xF0\x9F\x98\x82,0.2210,"[soo, funny, died, laughing, hahhahahahhah]",0.3769,0.5979
7,8,All language films are good. I like exploring ...,"[All, language, films, are, good., I, like, ex...",\xF0\x9F\x98\xB3,0.0180,"[language, films, good, like, e]",0.8016,0.8196
8,9,this @13Reasonswhy shouldnâ€™t end so soon. IM...,"[this, @13Reasonswhy, shouldnâ€™t, end, so, so...",\xF0\x9F\x98\x8A,0.6440,"[end, soon, imiss]",0.0000,0.6440
9,10,I need a break from all these social media mes...,"[I, need, a, break, from, all, these, social, ...",\xF0\x9F\x98\x8A,0.0000,"[need, break, social, media, mess, ever, done]",-0.3612,-0.3612


In [145]:
#Assigning score categories and logic
i = 0

predicted_value = [ ] #empty series to hold our predicted values

while(i<len(tweets)):
    if ((tweets.iloc[i]['final_sentimentvalue'] >= 0.1)):
        predicted_value.append('positive')
        i = i+1
    elif ((tweets.iloc[i]['final_sentimentvalue'] > -0.1) & (tweets.iloc[i]['final_sentimentvalue'] < 0.1)):
        predicted_value.append('neutral')
        i = i+1
    elif ((tweets.iloc[i]['final_sentimentvalue'] <= -0.1)):
        predicted_value.append('negative')
        i = i+1
tweets['predicted sentiment'] = predicted_value       
tweets.head(20)

,tweet_id,text,tokens,emoji,emoji_score,preprocessed_text,sentiment_score,final_sentimentvalue,predicted sentiment
0,1,. I really love the song. So relatable @ALIEGA...,"[., I, really, love, the, song., So, relatable...",\xF0\x9F\x98\x8D,0.4495,"[really, love, song, relatable, love]",1.2738,1.7233,positive
1,2,People should start watching the not mainstrea...,"[People, should, start, watching, the, not, ma...",\xF0\x9F\x98\x8A,0.6440,"[people, start, watching, mainstream, movies]",0.0000,0.6440,positive
2,3,I am so happy I am able to bring phone to my s...,"[I, am, so, happy, I, am, able, to, bring, pho...",\xF0\x9F\x98\xB3,0.0180,"[happy, able, bring, phone, school, happy]",1.1438,1.1618,positive
3,4,The actors in#PeakyBlinders are too good. I am...,"[The, actors, in#PeakyBlinders, are, too, good...",\xF0\x9F\x98\x8A,0.6440,"[actors, peakyblinders, good, e, next, season, e]",0.4404,1.0844,positive
4,5,The way @RyanMcMullan @ryavanhhuu__ sings are ...,"[The, way, @RyanMcMullan, @ryavanhhuu__, sings...",\xF0\x9F\x98\x8A,0.6440,"[way, sings, heavenly, people, listen, waytooggo]",0.6124,1.2564,positive
5,6,CANT WAIT To WATCH @RIVERDALE NEW SEASON! \xF0...,"[CANT, WAIT, To, WATCH, @RIVERDALE, NEW, SEASO...",\xF0\x9F\x98\x8A,0.6440,"[cant, wait, watch, new, season]",0.0000,0.6440,positive
6,7,him @CHRISPRATFANFOREVER45 is soo funny !!! I ...,"[him, @CHRISPRATFANFOREVER45, is, soo, funny, ...",\xF0\x9F\x98\x82,0.2210,"[soo, funny, died, laughing, hahhahahahhah]",0.3769,0.5979,positive
7,8,All language films are good. I like exploring ...,"[All, language, films, are, good., I, like, ex...",\xF0\x9F\x98\xB3,0.0180,"[language, films, good, like, e]",0.8016,0.8196,positive
8,9,this @13Reasonswhy shouldnâ€™t end so soon. IM...,"[this, @13Reasonswhy, shouldnâ€™t, end, so, so...",\xF0\x9F\x98\x8A,0.6440,"[end, soon, imiss]",0.0000,0.6440,positive
9,10,I need a break from all these social media mes...,"[I, need, a, break, from, all, these, social, ...",\xF0\x9F\x98\x8A,0.0000,"[need, break, social, media, mess, ever, done]",-0.3612,-0.3612,negative


In [146]:
# import the relevant libraries
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
import string
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [147]:
# data preprocessing (pt1)
#function to work with Regex
def remove_pattern(input_text, pattern):
    r = re.findall(pattern, input_text)
    for i in r:
        input_text = re.sub(i, '', input_text)
    
    return input_text

# removing twitter handles (e.g @username)
tweets['text'] = np.vectorize(remove_pattern)((tweets['text']), "@[\w]*")


# removing hastags
tweets['text'] = np.vectorize(remove_pattern)(tweets['text'], "#[\w]*")


# remove blank rows if any
tweets['text'].dropna(inplace = True)


# change text to lowercase
tweets['text'] = [entry.lower() for entry in tweets['text']]


# tokenizing the text
tweets['text'] = [word_tokenize(entry) for entry in tweets['text']]

In [148]:
tweets.head(10)

,tweet_id,text,tokens,emoji,emoji_score,preprocessed_text,sentiment_score,final_sentimentvalue,predicted sentiment
0,1,"[., i, really, love, the, song, ., so, relatab...","[., I, really, love, the, song., So, relatable...",\xF0\x9F\x98\x8D,0.4495,"[really, love, song, relatable, love]",1.2738,1.7233,positive
1,2,"[people, should, start, watching, the, not, ma...","[People, should, start, watching, the, not, ma...",\xF0\x9F\x98\x8A,0.6440,"[people, start, watching, mainstream, movies]",0.0000,0.6440,positive
2,3,"[i, am, so, happy, i, am, able, to, bring, pho...","[I, am, so, happy, I, am, able, to, bring, pho...",\xF0\x9F\x98\xB3,0.0180,"[happy, able, bring, phone, school, happy]",1.1438,1.1618,positive
3,4,"[the, actors, in, are, too, good, ., i, am, ex...","[The, actors, in#PeakyBlinders, are, too, good...",\xF0\x9F\x98\x8A,0.6440,"[actors, peakyblinders, good, e, next, season, e]",0.4404,1.0844,positive
4,5,"[the, way, sings, are, so, heavenly, ., people...","[The, way, @RyanMcMullan, @ryavanhhuu__, sings...",\xF0\x9F\x98\x8A,0.6440,"[way, sings, heavenly, people, listen, waytooggo]",0.6124,1.2564,positive
5,6,"[cant, wait, to, watch, new, season, !, \xf0\x...","[CANT, WAIT, To, WATCH, @RIVERDALE, NEW, SEASO...",\xF0\x9F\x98\x8A,0.6440,"[cant, wait, watch, new, season]",0.0000,0.6440,positive
6,7,"[him, is, soo, funny, !, !, !, i, died, laughi...","[him, @CHRISPRATFANFOREVER45, is, soo, funny, ...",\xF0\x9F\x98\x82,0.2210,"[soo, funny, died, laughing, hahhahahahhah]",0.3769,0.5979,positive
7,8,"[all, language, films, are, good, ., i, like, ...","[All, language, films, are, good., I, like, ex...",\xF0\x9F\x98\xB3,0.0180,"[language, films, good, like, e]",0.8016,0.8196,positive
8,9,"[this, shouldnâ€™t, end, so, soon, ., imiss, t...","[this, @13Reasonswhy, shouldnâ€™t, end, so, so...",\xF0\x9F\x98\x8A,0.6440,"[end, soon, imiss]",0.0000,0.6440,positive
9,10,"[i, need, a, break, from, all, these, social, ...","[I, need, a, break, from, all, these, social, ...",\xF0\x9F\x98\x8A,0.0000,"[need, break, social, media, mess, ever, done]",-0.3612,-0.3612,negative


In [149]:
#importing nltk
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [150]:
# data preprocessing (pt2)
# removing Stop words, Non-Numeric and perfom Word Stemming/Lemmatization
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(tweets['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    tweets.loc[index,'text_final'] = str(Final_words)

In [151]:
tweets['traintestset'] = (tweets[['text_final','emoji']].agg(','.join, axis=1))  
tweets.head()

,tweet_id,text,tokens,emoji,emoji_score,preprocessed_text,sentiment_score,final_sentimentvalue,predicted sentiment,text_final,traintestset
0,1,"[., i, really, love, the, song, ., so, relatab...","[., I, really, love, the, song., So, relatable...",\xF0\x9F\x98\x8D,0.4495,"[really, love, song, relatable, love]",1.2738,1.7233,positive,"['really', 'love', 'song', 'relatable', 'love']","['really', 'love', 'song', 'relatable', 'love'..."
1,2,"[people, should, start, watching, the, not, ma...","[People, should, start, watching, the, not, ma...",\xF0\x9F\x98\x8A,0.6440,"[people, start, watching, mainstream, movies]",0.0000,0.6440,positive,"['people', 'start', 'watch', 'mainstream', 'mo...","['people', 'start', 'watch', 'mainstream', 'mo..."
2,3,"[i, am, so, happy, i, am, able, to, bring, pho...","[I, am, so, happy, I, am, able, to, bring, pho...",\xF0\x9F\x98\xB3,0.0180,"[happy, able, bring, phone, school, happy]",1.1438,1.1618,positive,"['happy', 'able', 'bring', 'phone', 'happy']","['happy', 'able', 'bring', 'phone', 'happy'],\..."
3,4,"[the, actors, in, are, too, good, ., i, am, ex...","[The, actors, in#PeakyBlinders, are, too, good...",\xF0\x9F\x98\x8A,0.6440,"[actors, peakyblinders, good, e, next, season, e]",0.4404,1.0844,positive,"['actor', 'good', 'excite', 'next', 'season']","['actor', 'good', 'excite', 'next', 'season'],..."
4,5,"[the, way, sings, are, so, heavenly, ., people...","[The, way, @RyanMcMullan, @ryavanhhuu__, sings...",\xF0\x9F\x98\x8A,0.6440,"[way, sings, heavenly, people, listen, waytooggo]",0.6124,1.2564,positive,"['way', 'sings', 'heavenly', 'people', 'listen']","['way', 'sings', 'heavenly', 'people', 'listen..."


In [152]:
#download clean data if needed
tweets.to_csv(r'cleandata.csv', index = False)

Part 2-Section B
Building Naive Bayes Classifier

In [153]:
# train and test dataset split

tweets['traintestset']= tweets['traintestset'].astype(str)
train_x, test_x, train_y, test_y = model_selection.train_test_split(tweets['traintestset'], 
                                tweets['predicted sentiment'],
                                test_size = 0.3)
# encoding the dataset
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [154]:
# word vectorization

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(tweets['traintestset'])
Train_X_Tfidf = Tfidf_vect.transform(train_x)
Test_X_Tfidf = Tfidf_vect.transform(test_x)


In [155]:
# classification using Naive Bayes classifier

Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,train_y)

# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
#print(Test_X_Tfidf)
#print(predictions_NB)

In [156]:
#Testing Naive Bayes Model
Naive.score(Test_X_Tfidf,test_y)

0.7402597402597403

In [157]:
#Formulation of confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = Naive.fit(Train_X_Tfidf,train_y).predict(Test_X_Tfidf)
cm = confusion_matrix(test_y, y_pred)
print (cm)

[[30 13  4]
 [ 1 39  5]
 [ 3 14 45]]


In [158]:
#Precision and recall
def precision(label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()
    
def recall(label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()

def precision_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_precisions = 0
    for label in range(rows):
        sum_of_precisions += precision(label, confusion_matrix)
    return sum_of_precisions / rows

def recall_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_recalls = 0
    for label in range(columns):
        sum_of_recalls += recall(label, confusion_matrix)
    return sum_of_recalls / columns

In [159]:
#Precision and recall for each label
print("label precision recall")
for label in range(3):
    print(f"{label:5d} {precision(label, cm):9.3f} {recall(label, cm):6.3f}")

label precision recall
    0     0.882  0.638
    1     0.591  0.867
    2     0.833  0.726


In [160]:
#Precision and Recall of the Naive Bayes Model
print("precision total:", precision_macro_average(cm))

print("recall total:", recall_macro_average(cm))

precision total: 0.7688651218062983
recall total: 0.7435903302066652


In [161]:
#Accuracy of model based on confusion matrix
def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements = confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements 
accuracy(cm)

0.7402597402597403

In [162]:
#Get the accuracy of the model
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, test_y)*100)

Naive Bayes Accuracy Score ->  74.02597402597402


In [163]:
#Save the saved Naive Bayes Model
import pickle
f = open('naivebayesclassifier.pickle', 'wb')
pickle.dump(Naive , f)
f.close()